In [32]:
import os
from itertools import chain

fpath_win = 'C:\\Users\\rwmil\\Documents\\GitHub\\llm\\llm\\'
fpath_mac = '/Users/richardmiller/Documents/GitHub/llm/'
os.chdir(fpath_mac)
print(os.getcwd())

/Users/richardmiller/Documents/GitHub/llm


In [ ]:
with open('./LLMs-from-scratch-main/ch02/01_main-chapter-code/the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()
f.close()

split_text = [re.split(r'([,.:;?_!"()\']|--|\s)',i) for i in raw_text.split()]
split_text = [i.strip() for item in split_text for i in item if i not in ['',' ']]
split_text.extend(['<|endoftext|>','<|unk|>'])

split_text = sorted(list(set(split_text)))
vocab = {token:i for i,token in enumerate(split_text)}

In [ ]:
class SimpleTokenizer:
    def __init__(self,vocab):
        ''' The vocab object must be a dictionary or 
        dictionary-like object and must be defined 
        outside of the tokenizer class. It also must
        have the form of {string:integer}'''
        self.string_to_int = vocab
        self.int_to_string = {i:s for s,i in vocab.items()}
        self.re_string_encoder = r'([,.:;?_!"()\']|--|\s)' #Leave these here because I can use them later to define the regex
        self.re_string_decoder = r'\s+([,.:;?!"()\'])' # strings that I want to use later. If my texts become more complex.

    def encoder(self,text):
        preprocess = re.split(self.re_string_encoder,text)
        preprocess = [i.strip() for i in preprocess if i not in ['',' ']]
        preprocess = [i if i in self.string_to_int else '<|unk|>' for i in preprocess]
        
        
        return [self.string_to_int[s] for s in preprocess]

    def decoder(self,ids):
        text = ' '.join([self.int_to_string[i] for i in ids])
        text = re.sub(self.re_string_decoder,r'\1',text)
        return text

tokenizer = SimpleTokenizer(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
te = " <|endoftext|> ".join((text1, text2))
out = tokenizer.encoder(te)
out2 = tokenizer.decoder(out)

In [ ]:
print(out2)

In [ ]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

In [ ]:
tokenizer = tiktoken.get_encoding('gpt2')
text = ('Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.')
integers = tokenizer.encode(text,allowed_special={'<|endoftext|>'})
print(integers)

In [ ]:
strings = tokenizer.decode(integers)
print(strings)

## Section 2 Using Byte Pair Encoding

In [ ]:
import os
import tiktoken

fpath_win = 'C:\\Users\\rwmil\\Documents\\GitHub\\llm\\llm\\'
fpath_mac = '/Users/richardmiller/Documents/GitHub/llm/'
os.chdir(fpath_mac)
print(os.getcwd())

In [ ]:
#Get text data
with open('./LLMs-from-scratch-main/ch02/01_main-chapter-code/the-verdict.txt') as f:
    raw_text = f.read()
    
f.close()

#Initialize tokenizer and encode raw text from file
tokenizer = tiktoken.get_encoding('gpt2')
encoded_text = tokenizer.encode(raw_text)

#Remove last 50 characters
encoded_sample = encoded_text[:50]


In [ ]:
#Split encoded sample into x and y targets

context_size = 4 #Determines number of tokens included in input
x = encoded_sample[:context_size]
y = encoded_sample[1:context_size+1]

for i in range(1,context_size+1):
    context = encoded_sample[:i]
    desired = encoded_sample[i]
    print(context,'----->',desired)
    

In [ ]:
for i in range(1,context_size+1):
    context = encoded_sample[:i]
    desired = encoded_sample[i]
    print(tokenizer.decode(context),'----->',tokenizer.decode([desired]))

## Section 3

In [33]:
import torch
from torch.utils.data import DataLoader, Dataset
import tiktoken


In [34]:

class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        '''
        Looks like tokenizer is instantiated before being passed to this class.
        '''
        
        self.input_ids = []
        self.target_ids = []
        
        #Get token id's
        self.token_ids = tokenizer.encode(txt)
        
        for i in range(0,len(self.token_ids)-max_length,stride):
            input_chunk = self.token_ids[i:i+max_length]
            target_chunk = self.token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids, self.target_ids
    
    
        

In [35]:
def create_dataloader_v1(
    txt,
    batch_size=25,
    max_length=255,
    stride=128,
    shuffle=True,
    drop_last=True,
    num_workers=0,
):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    
    return dataloader



In [36]:
with open('./LLMs-from-scratch-main/ch02/01_main-chapter-code/the-verdict.txt') as f:
    raw_data = f.read()
    
f.close()

#max_length and stride can be changed, using a larger stride will
#result in less overfitting

#batch_size is a tunable hyperparameter to make the model better.
#Higher batch_size means more memory usage and less noise
#Lower batch_size means less memory usage but more nosie in training

dataloader = create_dataloader_v1(
    raw_data,
    batch_size=1,
    max_length=4,
    stride=1,
    shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)


In [37]:
input_ids = torch.tensor([2,3,5,1])
vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size,output_dim)



In [74]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)
max_length = 4
dataloader = create_dataloader_v1(raw_data,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter = iter(dataloader)
inputs,targets = next(data_iter)

token_embeddings = token_embedding_layer(inputs[1])

context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)
pos_embedding = pos_embedding_layer(torch.arange(context_length))
print(pos_embedding.shape,token_embedding.shape)

torch.Size([4, 256]) torch.Size([8, 4, 256])


In [68]:
input_embeddings = token_embeddings+pos_embedding

# Chapter 3 - Self-Attention

In [82]:
import torch
inputs = torch.tensor([
    [0.43,0.15,0.89],
    [0.55,0.87,0.66],
    [0.57,0.85,0.64],
    [0.22,0.58,0.33],
    [0.77,0.25,0.10],
    [0.05,0.80,0.55],
])

query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i,query)
    
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [83]:
attn_weights_2_tmp = attn_scores_2/attn_scores_2.sum()
print('Attention Weights: ',attn_weights_2_tmp)
print('Sum: ',attn_weights_2_tmp.sum())

Attention Weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum:  tensor(1.0000)


In [85]:
#Computing attention weights of a single token by scalar multiplication of a vector
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
    
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [87]:
#Computing attention weights of all tokens by dot product
attn_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x_i,x_j)
        
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [88]:
attn_scores = inputs@inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [89]:
attn_weights = torch.softmax(attn_scores,dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


## Chatper 3 Section 4.1

In [7]:
import torch
inputs = torch.tensor([
    [0.43,0.15,0.89],
    [0.55,0.87,0.66],
    [0.57,0.85,0.64],
    [0.22,0.58,0.33],
    [0.77,0.25,0.10],
    [0.05,0.80,0.55],
])

#Define input element, input embadding size, and output embedding size
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

#Initialize Weight tensors
#requires_grad is set to false to reduce output clutter, but during model training it needs to be true
torch.manual_seed(123) #For repeatability
W_query = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad=False)


query_2 = x_2@W_query
key_2 = x_2@W_key
value_2 = x_2@W_value

#Weight Tensors are optimized during training and are used to make predictions
#Attention Weights are tensors that give the amount of attention to different parts of
# the input. Attention weights are like saying since this word directly preceeds the unknown I
# will pay more attention to it than I will to another word that is 40 tokens away.

#Get all keys and values:
keys = inputs@W_key
values = inputs@W_value


In [15]:
#Computing an individual weight
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print('Attention score for position 2: ',attn_score_22)

#Compute all weights using tensor dot product
attn_scores_2 = query_2@keys.T
print('Attention Scores: ',attn_scores_2)

#Notice that the second value in attn_scores_2 matches the value of attn_score_22

#Calculate attention scores from attention weights
#To help normalize we are dividing by the square root
#I think this is going to be like the standard deviation where
# you divide by sqrt(N) for a population.
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2/d_k**0.5, dim=-1)
print('Scaled Attention Weights: ',attn_weights_2)

#Calculate context vector
context_vect_2 = attn_weights_2 @ values
print('Context Vector: ',context_vect_2)

#Query refers to the current item (token, word, sentance, etc.)
#Key is the indexing value of a token and is used to match the query
#Value the actual content or input values

Attention score for position 2:  tensor(1.8524)
Attention Scores:  tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])
Scaled Attention Weights:  tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
Context Vector:  tensor([0.3061, 0.8210])


## Chapter 3 Section 4.2

In [20]:
#It's useful to organize all the code in Chapter 3 Section 4.1 into a single
# succinct python class for use in the LLM architecture.

import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self,d_in,d_out):
        '''
        Initializes weight tensors for queries, values, and keys.
        Transforms from input dimension, d_in, to output dimension, d_out.
        '''
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in,d_out))
        self.W_key = nn.Parameter(torch.rand(d_in,d_out))
        self.W_value = nn.Parameter(torch.rand(d_in,d_out))
        
    def forward(self,x):
        '''
        Compute keys, queries, and values tensors by cross product with their respective weights.
        Calculate attention scores.
        Normalize attention scores with softmax to get attention weight tensor.
        Create a context vector.
        
        '''
        keys = x@self.W_key
        queries = x@self.W_query
        values = x@self.W_value
        
        attn_scores = queries@keys.T
        attn_weights = torch.softmax(
            attn_scores/keys.shape[-1]**0.5, 
            dim=-1
        )
        
        context_vect = attn_weights@values
        return context_vect
    
    
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in,d_out)
print(sa_v1(inputs))

#NOTE: using nn.Linear layers will perform the matrix multiplication and with optimized weight initializations.
# This should result in quicker and more reliable training versus using randomized weights.

#Will impliment this in SelfAttention_v2

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [21]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        
        self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
    def forward(self, x):
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        
        attn_scores = queries@keys.T
        attn_weights = torch.softmax(
            attn_scores/keys.shape[-1]**0.5,
            dim=-1
        )
        
        context_vect = attn_weights@values
        
        return context_vect
    
    
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in,d_out)

print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


# Chapter 3 Section 5

In [ ]:
#Casual Self-Attention (AKA Masked Attention): Prevents a model from learning words after the token word. 
#Because the model won't have access to information after some token. Restricts the model's ability to learn
# from the previous data before the token.

#Multi-Head Self-Attention: Splits the Self-Attention mechanism into multiple sub-units or 'heads', this
# splits up the learning process. Each head learns from different parts of the input text or different contexts.
# This allows or increases the model's ability to perform complex tasks.

#Consider the case of data represented as a tensor, T. In 2-dimensions, the current token data is the diagonal
# elements of the input array. So, the values above the diagonal are masked. In a multi-dimensional tensor, this
# would correspond to the diagonal hyper-plane. All elements in the volume above the diagonal hyper-plane
# will be masked. This prevents the model from accessing data after a given token and restricts it to data from
# before the given token.

#The unmasked weights are normalized to 1.



## Chapter 3 Section 5.1 - Applying a Casual Attention Mask

In [23]:
#Class from Chapter 3 Section 4.2

class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        
        self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
    def forward(self, x):
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        
        attn_scores = queries@keys.T
        attn_weights = torch.softmax(
            attn_scores/keys.shape[-1]**0.5,
            dim=-1
        )
        
        context_vect = attn_weights@values
        
        return context_vect
    
    
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in,d_out)

In [29]:
#Compute attention weights with the SelfAttention_v2 Class from
# Chapter 3 Section 4.2
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries@keys.T
attn_weights = torch.softmax(
    attn_scores/keys.shape[-1]**0.5,
    dim=-1
)

#Print the tensor we want to mask:
print('Attention Weights Tensor: ',attn_weights)

#Use tril() function to create a masking tensor with 0's in the upper triangle
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))

print('Simple Masking Tensor: ',mask_simple)

#Normalize Masking Tensor
row_sums = mask_simple.sum(dim=-1,keepdim=True)
mask_simple_norm = mask_simple/row_sums
print('Normalized Masking Tensor: ', mask_simple_norm)

#Masking and renormalizing will prevent upper triangular data from the softmax from leaking into
# the masking.

Attention Weights Tensor:  tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
Simple Masking Tensor:  tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
Normalized Masking Tensor:  tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0

In [34]:
#Improving on the masking function

#Softmax converts the data into a Poisson probability distribution. We can represent zeros in the upper
# triangle matrix as e^-inf. This can be beneficial to us. If we replace the upper triangle of the tensor with -inf
# and then take the softmax, it will become an upper triangle of masked zeros.

#Define an improved masking tensor:
mask = torch.triu(torch.ones(context_length,context_length),diagonal=1) #diagonal=1 starts at the line above the diagonal
masked = attn_scores.masked_fill(mask.bool(),-torch.inf) #Uses the masked_fill method in pytorch to fill the upper triangle with -infinity

print('Masking Tensor: ',masked)

#Make Attention Weights tensor
attn_weights = torch.softmax(masked/keys.shape[-1]**0.5, dim=1)
print('Attention Weights: ',attn_weights)

#Calculate Context Tensor:
context_vect = attn_weights@values
print('Context Tensor: ', context_vect)

Masking Tensor:  tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)
Attention Weights:  tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
Context Tensor:  tensor([[0.1855, 0.8812],
        [0.2795, 0.9361],
        [0.3133, 0.9508],
        [0.2994, 0.8595],
        [0.2702, 0.7554],
        [0.2772, 0.7618]], grad_fn=<MmBackwar

## Chapter 3 Section 5.2

In [37]:
#Dropout can be used to help prevent overfitting to available data and increase general usefulness of
# your model. It does this by ignoring hidden layer units.

#Dropout is only used during training and is disabled for model predictions.

#For transformer architectures, dropout is typically employed at two points in the training process.
# 1. After calculating attention weights
# 2. After applying attention weights to value vectors

#Here we will use a dropout rate of 50%, but typically dropout rates are 10% to 20% (0.1 to 0.2)

#Apply the dropout to a 6x6 tensor for illustration purposes:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
print(dropout(example))

#To account for half of the values being dropped out, the other values are doubled.

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [39]:
#Apply the dropout method to the attention weights:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


## Chapter 3 Section 5.3 - Implementing a Compact Casual Attention Class

In [44]:
#Impliment a CasualAttention class by modifying the SelfAttention class. This will then be used
# later as a template for the MultiHeadAttion class that we will code.

class CasualAttention(nn.Module):
    def __init__(
        self,
        d_in,
        d_out,
        context_length,
        dropout,
        qkv_bias = False,
    ):
        super().__init__()
        
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length,context_length), diagonal=1),
        )
        
    def forward(self,x):
        b,num_tokens,d_in = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        
        attn_scores = queries@keys.transpose(1,2)
        attn_scores.masked_fill(
            self.mask.bool()[:num_tokens,:num_tokens],
            -torch.inf
        )
        attn_weights = torch.softmax(
            attn_scores/keys.shape[-1]**0.5,
            dim=-1
        )
        
        attn_weights = self.dropout(attn_weights)
        
        context_vect = attn_weights@values
        
        return context_vect
    

In [45]:
#Make a batched set of identical data inputs:
batch = torch.stack((inputs,inputs), dim=0)

torch.manual_seed(123)
context_length = batch.shape[-1]
ca = CasualAttention(d_in, d_out, context_length, 0.0)
context_vect = ca(batch)
print('Context Tensor: ',context_vect)

RuntimeError: The size of tensor a (3) must match the size of tensor b (6) at non-singleton dimension 2